In [ ]:
'''
import asyncio
import websockets
import os

# 利用ffmpeg库将图片生成视频  帧率:5  h264编码
def convert_jpg_to_mp4(jpg_path, output_name):
    os.system(r"ffmpeg -y -framerate 5 -i {}\%d.jpg "
              r"D:\plantarPressureTest\webSocket\photo\{}.avi".format(jpg_path, output_name))
    return True

async def send_video_file(websocket, video_file_path):
    # Open the video file in binary mode
    with open(video_file_path, 'rb') as video_file:
        while True:
            # Read a chunk of data from the video file
            data_chunk = video_file.read(1024)
            if not data_chunk:
                break
            # Send the data chunk to the WebSocket client
            await websocket.send(data_chunk)

async def echo(websocket, path):
    # fetch msg
    async for message in websocket:
        print("got a message: {}".format(message))
        await websocket.send(message)

async def main():
    jpg_path = r"D:/plantarPressureTest/webSocket/photo"
    output_name = r"abc"

    # Convert jpg to mp4
    convert_jpg_to_mp4(jpg_path, output_name)

    # Video file path
    video_file_path = os.path.join(jpg_path, "{}.avi".format(output_name))

    # start a WebSocket server
    async with websockets.serve(echo, "10.1.48.166", 8765):
        # Wait for a client to connect
        async with websockets.connect("ws://10.1.48.166:8765") as websocket:
            # Send the video file data to the WebSocket client
            await send_video_file(websocket, video_file_path)
            await asyncio.Future()  # run forever

await main()
'''

import asyncio
import websockets
import os

async def send_video(websocket, video_path):
    # Check if the video file exists
    if not os.path.exists(video_path):
        print("Video file not found: {}".format(video_path))
        return

    # Open the video file in binary mode
    with open(video_path, 'rb') as video_file:
        try:
            while True:
                # Read a chunk of data from the video file
                data_chunk = video_file.read(1024)
                if not data_chunk:
                    break
                # Send the data chunk to the WebSocket client
                await websocket.send(data_chunk)
        # 处理异常情况
        except websockets.exceptions.ConnectionClosedOK:
            # Connection closed by the client
            print("Client disconnected.")
        except Exception as e:
            print("An error occurred while sending video: {}".format(e))

async def handle_client(websocket, path):
    print("Client connected.")
    video_path = "D:/plantarPressureTest/webSocket/photo/abc.avi"  # 替换为视频文件的路径
    await send_video(websocket, video_path)

async def main():
    server_host = "10.1.48.166"  # WebSocket服务器的IP地址
    server_port = 8765  # WebSocket服务器的端口号

    # 启动WebSocket服务器
    server = await websockets.serve(handle_client, server_host, server_port)

    print("WebSocket server started at ws://{}:{}".format(server_host, server_port))

    # 等待服务器关闭，服务器运行期间，它会一直阻塞在这里，直到服务器关闭
    await server.wait_closed() 

if __name__ == "__main__":
    await main() # 确保服务器运行时不会退出，一直保持监听状态，接收来自客户端的连接














